In [4]:
!pip install -q haystack-ai sentence-transformers torch

from haystack import Document, Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.embedders import SentenceTransformersDocumentEmbedder, SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.generators import HuggingFaceLocalGenerator
from haystack.components.builders import PromptBuilder
import torch

# 1. Configuració bàsica
document_store = InMemoryDocumentStore()

# 2. Carrega els teus documents mèdics (exemple)
documents = [
    Document(content="La insuficiència renal crònica es diagnostica amb anàlisis de creatinina i TFG."),
    Document(content="Els símptomes inclouen fatiga, nàusees i retenció de líquids.")
]

# 3. Embeddings eficients
doc_embedder = SentenceTransformersDocumentEmbedder(model="BAAI/bge-small-en-v1.5")
doc_embedder.warm_up()
docs_with_embeddings = doc_embedder.run(documents)
document_store.write_documents(docs_with_embeddings["documents"])

# 4. Model lleuger (Phi-2 2.7B)
from haystack.components.generators import HuggingFaceLocalGenerator
from haystack.utils.device import ComponentDevice

'''generator = HuggingFaceLocalGenerator(
    model="microsoft/phi-2",
    device="cuda"  # o "cpu" si no tens GPU
    #device_map="auto",
    #torch_dtype=torch.float16,
    #generation_kwargs={"max_new_tokens": 150}
)'''

'''generator = HuggingFaceLocalGenerator(
    model="microsoft/phi-2",
    device=ComponentDevice.from_str("cpu")  # o "cpu"
)

generator.warm_up()'''

generator = HuggingFaceLocalGenerator(
    model="google/flan-t5-base",
    device=ComponentDevice.from_str("cpu")
)
generator.warm_up()

# 5. Pipeline RAG simplificat
prompt_template = """
Respon la pregunta basant-te únicament en aquesta informació:
{% for doc in documents %}
{{ doc.content }}
{% endfor %}

Pregunta: {{ query }}
Resposta:"""

rag_pipe = Pipeline()
rag_pipe.add_component("text_embedder", SentenceTransformersTextEmbedder(model="BAAI/bge-small-en-v1.5"))
rag_pipe.add_component("retriever", InMemoryEmbeddingRetriever(document_store, top_k=2))
rag_pipe.add_component("prompt_builder", PromptBuilder(template=prompt_template, required_variables=["documents", "query"]))
rag_pipe.add_component("llm", generator)

rag_pipe.connect("text_embedder", "retriever")
rag_pipe.connect("retriever", "prompt_builder.documents")
rag_pipe.connect("prompt_builder", "llm")

# 6. Execució
query = "Com es diagnostica la insuficiència renal crònica?"
result = rag_pipe.run({"text_embedder": {"text": query}, "prompt_builder": {"query": query}})
print(result["llm"]["replies"][0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Device set to use cpu


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

amb anàlisis de creatina i TFG


In [5]:
!pip install -q haystack-ai sentence-transformers torch
!pip install biopython

from haystack import Document, Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.embedders import SentenceTransformersDocumentEmbedder, SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.generators import HuggingFaceLocalGenerator
from haystack.components.builders import PromptBuilder
import torch

# 1. Configuració bàsica
document_store = InMemoryDocumentStore()

# 2. Carrega els teus documents mèdics (exemple)
'''documents = [
    Document(content="La insuficiència renal crònica es diagnostica amb anàlisis de creatinina i TFG."),
    Document(content="Els símptomes inclouen fatiga, nàusees i retenció de líquids.")
]
'''
#from langchain_core.documents import Document

from Bio import Entrez

# Necessari per a l'API de PubMed
Entrez.email = "alba.figueras@estudiantat.upc.edu"

# Cerquem els articles més recents sobre "cardiology guidelines 2023"
handle = Entrez.esearch(db="pubmed", term="cardiology guidelines 2023", retmax=5)
record = Entrez.read(handle)
ids = record["IdList"]

# Ara recuperem els resums d’aquests articles
handle = Entrez.efetch(db="pubmed", id=",".join(ids), rettype="abstract", retmode="text")
abstracts = handle.read()
#print(abstracts)

# Separem cada resum en un document
texts = abstracts.strip().split("\n\n")
texts = texts[:5]

from haystack import Document  # <-- Això n'hi ha prou

# Modifica la creació de documents així:
documents = [Document(content=text.strip()) for text in texts[:5]]  # content en lloc de page_content

#documents = [Document(page_content=t.strip()) for t in texts]

# 3. Embeddings eficients
#doc_embedder = SentenceTransformersTextEmbedder(model="BAAI/bge-small-en-v1.5")
doc_embedder = SentenceTransformersDocumentEmbedder(model="BAAI/bge-small-en-v1.5")
doc_embedder.warm_up()
docs_with_embeddings = doc_embedder.run(documents)
document_store.write_documents(docs_with_embeddings["documents"])

# 4. Model lleuger (Phi-2 2.7B)
from haystack.components.generators import HuggingFaceLocalGenerator
from haystack.utils.device import ComponentDevice

'''generator = HuggingFaceLocalGenerator(
    model="microsoft/phi-2",
    device="cuda"  # o "cpu" si no tens GPU
    #device_map="auto",
    #torch_dtype=torch.float16,
    #generation_kwargs={"max_new_tokens": 150}
)'''

'''generator = HuggingFaceLocalGenerator(
    model="microsoft/phi-2",
    device=ComponentDevice.from_str("cpu")  # o "cpu"
)

generator.warm_up()'''

generator = HuggingFaceLocalGenerator(
    model="google/flan-t5-base",
    device=ComponentDevice.from_str("cpu")
)
generator.warm_up()

# 5. Pipeline RAG simplificat
prompt_template = """
Ets un assistent mèdic especialitzat en nefrologia.
Respon la pregunta únicament si trobes informació rellevant als documents.
Si no hi ha informació, respon "No s'ha trobat informació rellevant".

Documents:
{% for doc in documents %}
- {{ doc.content }}
{% endfor %}

Pregunta: {{ query }}
Resposta:"""

rag_pipe = Pipeline()
rag_pipe.add_component("text_embedder", SentenceTransformersTextEmbedder(model="BAAI/bge-small-en-v1.5"))
rag_pipe.add_component("retriever", InMemoryEmbeddingRetriever(document_store, top_k=2))
rag_pipe.add_component("prompt_builder", PromptBuilder(template=prompt_template, required_variables=["documents", "query"]))
rag_pipe.add_component("llm", generator)

rag_pipe.connect("text_embedder", "retriever")
rag_pipe.connect("retriever", "prompt_builder.documents")
rag_pipe.connect("prompt_builder", "llm")

# 6. Execució
result = rag_pipe.run({
    "text_embedder": {"text": query},
    "prompt_builder": {"query": query}
})

if "no answer" in result["llm"]["replies"][0].lower():
    print("⚠️ No s'ha trobat informació suficient als documents")
else:
    print("💡 Resposta:", result["llm"]["replies"][0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Device set to use cpu


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

💡 Resposta: Yangjyhd@qdu.edu.cn.
